## Are a single reviewer's scores autocorrelated?  
  
Procedure:
* Plot, for randomly selected (20? 30?) reviewers with over 20 reviews, their consecutive scores against review #.
    * Do the same for another subset's BNM awarding behavior
* Look at average time lag between reviews (i.e. if most reviewers review an album on avg every week, we'd let time lag 1 be equivalent to 7 days)
* Visualize partial autocorrelations to see which lag is appropriate for autocorr calculations
* Compute autocorrelations:
    * For each author, compute autocorrelation using determine time lag
    * Use 1-sample t to see if on avg the autocorrs are significantly diff from 0

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql
import os
from scipy.signal import savgol_filter
from scipy import stats

pd.set_option('precision', 2)
np.set_printoptions(precision=2)

plt.rcParams['axes.facecolor'] = '0.95'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        db_path = os.path.join(dirname, filename)

# connect to SQL database, create connection object to database
connection = sql.connect(db_path)
print("SQL database connected")

table = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", connection)

reviews = pd.read_sql('SELECT * FROM reviews', connection)
genres = pd.read_sql('SELECT * FROM genres', connection)
connection.close()
print('SQL database connection closed')

SQL database connected
SQL database connection closed


## Cleaning and augmenting data  
We're going to be sorting by review publication date, so let's convert it from str format to unix datetime for ease of use:

In [2]:
reviews['pub_date'] = pd.to_datetime(reviews.pub_date, format = '%Y-%m-%d')
reviews['unix_time'] = reviews.pub_date.view(np.int64) // 10**9  
"""
 Division by 10^9: 
 We truncate trailing zeros since the int represents seconds elapsed since new year's 1970! 
 However, we leave two trailing zeros (as opposed to dividing by 10^11) since conversion to days later on requires another division by 60*60(*24).

"""

"\n Division by 10^9: \n We truncate trailing zeros since the int represents seconds elapsed since new year's 1970! \n However, we leave two trailing zeros (as opposed to dividing by 10^11) since conversion to days later on requires another division by 60*60(*24).\n\n"

The subsequent analyses will require us to augment the reviews data with each review's number according to the author's history.

In [3]:
reviews_authns = reviews.copy() # review dataframe, augmented with each review's 'number' i.e. its chronology in author's review history
reviews_authns['review_num'] = pd.Series(index=reviews_authns.index, dtype='int64')
reviews_authns['days_since_prev'] = pd.Series(index=reviews_authns.index, dtype='int64')

for a, rows in reviews_authns.groupby('author'):
    ordered = rows.sort_values(by='unix_time')
    n = ordered.shape[0]
    
    # add review number per author
    nums = list(range(1, n+1))
    reviews_authns.at[rows.index, 'review_num'] = nums
    
    # add days since author's last review
    days = np.zeros(n)
    for i in nums[1:]:
        t_curr = ordered.iloc[i-1].unix_time
        t_prev = ordered.iloc[i-2].unix_time
        days[i-1] = (t_curr - t_prev) / (60*60*24) # convert difference from seconds to days
        
    days[0] = np.nan  # first reviews will not have a 'days since' value
    reviews_authns.loc[ordered.index, 'days_since_prev'] = days

Let's take a coarse look at the average reviewers' scoring behavior over the course of 20 reviews. We'll randomly choose 5 reviewers with 20 or more reviews from the dataset, then plot against the review number their a) scores and b) best new music awards.